In [1]:
#Importing Necessary Libraries.
from PIL import Image
import cv2
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout

Using TensorFlow backend.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['cell_images']


In [3]:
ls

__notebook__.ipynb  __output__.json


## Data Preparation
Data Preparation: We will make data and labels list where data will be image to array implementation which contains RGB values of each image and label will be class of cellls here I will be taking 0 and 1 for two classes.

In [4]:
data = []
labels = []
Parasitized = os.listdir("../input/cell_images/cell_images/Parasitized/")
for a in Parasitized:
    try:
        image = cv2.imread("../input/cell_images/cell_images/Parasitized/"+a)
        image_from_array = Image.fromarray(image, 'RGB')
        size_image = image_from_array.resize((50, 50))
        data.append(np.array(size_image))
        labels.append(0)
    except AttributeError:
        print("")

Uninfected = os.listdir("../input/cell_images/cell_images/Uninfected/")
for b in Uninfected:
    try:
        image = cv2.imread("../input/cell_images/cell_images/Uninfected/"+b)
        image_from_array = Image.fromarray(image, 'RGB')
        size_image = image_from_array.resize((50, 50))
        data.append(np.array(size_image))
        labels.append(1)
    except AttributeError:
        print("")

In [5]:
Cells = np.array(data)
labels = np.array(labels)

Saving and Loading the data we prepared so next time we can load it from saved .npy file.

In [6]:
np.save("Cells",Cells)
np.save("labels",labels)

In [7]:
Cells = np.load("Cells.npy")
labels = np.load("labels.npy")

Do Train/Test Split of data and labels that prepared in early section. Classes are defined as the unique labels in the data. Here it will be 2 as Parasitized:0 and Uninfected:1, here 0 and 1 are the mapping in labels for these two classes

In [8]:
s = np.arange(Cells.shape[0])
np.random.shuffle(s)
Cells = Cells[s]
labels = labels[s]

In [9]:
num_classes = len(np.unique(labels))
len_data = len(Cells)

In [10]:
(x_train,x_test) = Cells[(int)(0.1*len_data):],Cells[:(int)(0.1*len_data)]
x_train = x_train.astype('float32')/255 # As we are working on image data we are normalizing data by divinding 255.
x_test = x_test.astype('float32')/255
train_len = len(x_train)
test_len = len(x_test)

In [11]:
(y_train, y_test) = labels[(int)(0.1*len_data):], labels[:(int)(0.1*len_data)]


## One hot encoding:
Here the problem has two classes so last output layer of neural network will have 2 neurons one for each class, One hot encoding will help us to change labels in binary format. example: 2 can be represented as [1 0] if output layer has 2 neurons and [0 0 1 0] if output has 4 neurons/classes


In [12]:
#Doing One hot encoding as classifier has multiple classes
y_train = keras.utils.to_categorical(y_train,num_classes)
y_test = keras.utils.to_categorical(y_test,num_classes)


## Create Sequential Model:
Here I will be using Relu{max(0,z)}, You can try tanh/sigmoid/Leaky Relu for finding performance on various activation functions.Our output layer will be softmax activation rather than sigmoid as we have more than one class to classify. softmax activation calculates e^value/sum(all_values_in_axis[0 or 1])


In [13]:
#creating sequential model
model=Sequential()
model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(50,50,3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(500,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(2,activation="softmax"))#2 represent output layer neurons 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 50, 50, 16)        208       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 25, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 25, 25, 32)        2080      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 64)        8256      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 6, 6, 64)          0         
__________

compile the model with loss as categorical_crossentropy and using adam optimizer you can test result by trying RMSProp as well as Momentum

In [14]:
# compile the model with loss as categorical_crossentropy and using adam optimizer you can test result by trying RMSProp as well as Momentum
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [15]:
#Fit the model with min batch size as 50[can tune batch size to some factor of 2^power ] 
model.fit(x_train, y_train, batch_size=50, epochs=20, verbose=1)

Epoch 1/20
24803/24803 [==============================] - 33s 1ms/step - loss: 0.4792 - acc: 0.7555
Epoch 2/20
24803/24803 [==============================] - 32s 1ms/step - loss: 0.1793 - acc: 0.9415
Epoch 3/20
24803/24803 [==============================] - 32s 1ms/step - loss: 0.1522 - acc: 0.9518
Epoch 4/20
24803/24803 [==============================] - 32s 1ms/step - loss: 0.1362 - acc: 0.9561
Epoch 5/20
24803/24803 [==============================] - 31s 1ms/step - loss: 0.1295 - acc: 0.9573
Epoch 6/20
24803/24803 [==============================] - 33s 1ms/step - loss: 0.1193 - acc: 0.9604
Epoch 7/20
24803/24803 [==============================] - 32s 1ms/step - loss: 0.1099 - acc: 0.9616
Epoch 8/20
24803/24803 [==============================] - 32s 1ms/step - loss: 0.1039 - acc: 0.9638
Epoch 9/20
24803/24803 [==============================] - 32s 1ms/step - loss: 0.0925 - acc: 0.9675
Epoch 10/20
24803/24803 [==============================] - 32s 1ms/step - loss: 0.0878 - acc: 0.9688

## Check the accuracy on Test data:¶

In [16]:
accuracy = model.evaluate(x_test, y_test, verbose=1)
print('\n', 'Test_Accuracy:-', accuracy[1])

2755/2755 [==============================] - 1s 528us/step

 Test_Accuracy:- 0.9600725952813067


## Save the model weights:

In [17]:
from keras.models import load_model
model.save('cells.h5')